# Required Libraries

In [ ]:
!pip install pandas
!pip install googletrans==4.0.0-rc1
!pip install unicode
!pip install pywin32
!pip install pycountry


## You must create a `.env` file containing the following paths:
### Carpeta_Excel_Raw = Path to the folder containing raw Excel files
### Carpeta_Nuevas_Extensiones = Folder where standardized files will be stored
### Carpeta_Normalizado = Folder where normalized files will be stored
### Carpeta_Consolidada = Folder where the consolidated file will be stored


# Standardizing files into XLSX format

In [ ]:
from pathlib import Path
import os
from win32com.client import Dispatch
import shutil
import pythoncom
from dotenv import load_dotenv

load_dotenv()
origen = Path(os.getenv("Carpeta_Excel_Raw", "")).resolve()
destino = Path(os.getenv("Carpeta_Nuevas_Extensiones", "")).resolve()
destino.mkdir(parents=True, exist_ok=True)

def convertir_y_copiar(origen: Path, destino: Path):
    pythoncom.CoInitialize()
    excel = Dispatch("Excel.Application")
    excel.DisplayAlerts = False  # <- Disables dialog boxes
    
    for archivo in origen.iterdir():
        if archivo.suffix.lower() == ".xls":
            ruta = str(archivo)
            nuevo = destino / (archivo.stem + ".xlsx")
            try:
                # Opens in read-only mode to avoid modifying the original
                wb = excel.Workbooks.Open(ruta, ReadOnly=True)
                # Saves as .xlsx
                wb.SaveAs(str(nuevo), FileFormat=51)
                # Closes without saving changes to the .xls
                wb.Close(SaveChanges=False)
                print(f"Converted: {archivo.name} → {nuevo.name}")
            except Exception as e:
                print(f"Error converting {archivo.name}: {e}")
        
        elif archivo.suffix.lower() == ".xlsx":
            try:
                shutil.copy2(str(archivo), str(destino / archivo.name))
                print(f"Copied: {archivo.name}")
            except Exception as e:
                print(f"Error copying {archivo.name}: {e}")
    
    excel.Quit()
    pythoncom.CoUninitialize()

# Runs the conversion
convertir_y_copiar(origen, destino)


# Data Normalization

In [ ]:
import pandas as pd
import os
import unicodedata
import pycountry_convert as pc
import pycountry  # To get ISO codes
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
entrada = os.getenv("Carpeta_Nuevas_Extensiones")
salida  = os.getenv("Carpeta_Normalizado")

# Function to detect Japanese characters
def contains_japanese(text):
    for ch in str(text):
        if ('\u4e00' <= ch <= '\u9fff') or ('\u3040' <= ch <= '\u309f') or ('\u30a0' <= ch <= '\u30ff'):
            return True
    return False

# Function to get continent from country using pycountry-convert
def obtener_continente(pais):
    try:
        country_code   = pc.country_name_to_country_alpha2(pais, cn_name_format="default")
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continents = {
            "AF": "Africa", "NA": "North America", "SA": "South America",
            "AS": "Asia",   "EU": "Europe",        "OC": "Oceania"
        }
        return continents.get(continent_code, "Unknown")
    except Exception:
        return "Unknown"

# Function to get ISO code of the country
def obtener_codigo_iso(pais):
    try:
        return pycountry.countries.lookup(pais).alpha_2
    except LookupError:
        return None

# Process each Excel file
for nombre_archivo in os.listdir(entrada):
    if not nombre_archivo.endswith(".xlsx"):
        continue
    try:
        print(f"Processing: {nombre_archivo}")
        ruta_entrada = os.path.join(entrada, nombre_archivo)
        nombre_base  = os.path.splitext(nombre_archivo)[0].replace('_Normalizado', '')
        ruta_salida  = os.path.join(salida, f"{nombre_base}_Normalizado.xlsx")

        # Read the file without headers
        df = pd.read_excel(ruta_entrada, header=None)

        # 1) Remove column A and row 1
        df = df.drop(columns=0).drop(index=0).reset_index(drop=True)

        # 2) Remove columns in Japanese (2 and 3)
        df = df.drop(columns=[2, 3], errors='ignore')

        # 3) Trim from column “合計” onwards (if it exists)
        encabezado0 = df.iloc[0].astype(str).apply(lambda x: unicodedata.normalize("NFKC", x))
        if "合計" in list(encabezado0):
            idx_total = list(encabezado0).index("合計")
            df = df.iloc[:, :idx_total]

        # 4) Ensure a maximum of 14 columns
        if df.shape[1] > 14:
            df = df.iloc[:, :14]

        # 5) Replace Japanese labels → English
        df = df.replace({'応募者': 'Applicant', '受験者': 'Examinee'})

        # 6) Prepare provisional headers
        encabezados = [unicodedata.normalize("NFKC", str(e)) for e in df.iloc[0].tolist()]

        # 7) Generate list of new column names
        niveles = ['N1', 'N2', 'N3', 'N4', 'N5']
        nuevas_columnas = ['Country/Region', 'City (ENG)']
        i = 2
        for nivel in niveles:
            if i + 1 < len(encabezados):
                nuevas_columnas += [f"{nivel} Applicants", f"{nivel} Examinees"]
                i += 2

        # 8) Assign new headers and remove the title row
        df.columns = nuevas_columnas
        df = df.drop(index=0).reset_index(drop=True)

        # 9) Clean Country/Region from Japanese text
        for idx, val in df['Country/Region'].items():
            if contains_japanese(val) and (idx + 1) in df.index:
                df.at[idx, 'Country/Region'] = unicodedata.normalize(
                    'NFKC', str(df.at[idx + 1, 'Country/Region'])
                )
        df['Country/Region'] = (
            df['Country/Region']
              .astype(str)
              .apply(lambda x: unicodedata.normalize('NFKC', x))
              .apply(lambda x: None if contains_japanese(x) else x)
        )
        df['Country/Region'] = df['Country/Region'].replace('nan', pd.NA).fillna(method='ffill')

        # 10) Filter valid rows: remove rows without City (ENG)
        df = df[df['Country/Region'].notna()]
        df = df[df['City (ENG)'].notna() & (df['City (ENG)'].astype(str).str.strip() != '')]

        # 11) Pivot to long format
        df_long = df.melt(
            id_vars=['Country/Region', 'City (ENG)'],
            var_name='metric',
            value_name='Count'
        )
        df_long[['Level', 'Type']] = df_long['metric'].str.split(' ', expand=True)
        df_long = df_long.drop(columns=['metric'])

        # 12) Add columns: Date, Year, Month
        parts = nombre_base.split('_')
        year = int(parts[0])
        mid = int(parts[1])
        fecha = pd.Timestamp(year, mid, 1).date()
        df_long['Fecha'] = fecha
        df_long['Año'] = year
        df_long['Mes'] = 'July' if mid == 1 else ('December' if mid == 2 else '')

        # Additional cleanup: remove extra whitespace
        df_long['Country/Region'] = df_long['Country/Region'].astype(str).str.strip()

        # 🆕 Manual replacements of problematic names (before getting ISO codes)
        reemplazos_paises = {
            "Brunei": "Brunei Darussalam",
            "Russia": "Russian Federation",
            "Turkey": "Türkiye",
            "Korea": "South Korea",
            "Ivory Coast": "Côte d'Ivoire",
            "Cote d' Ivoire": "Côte d'Ivoire",
            "Cote d'Ivoire": "Côte d'Ivoire",
            "DR Congo": "Congo, The Democratic Republic of the",
            "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
            "Mongol": "Mongolia",
            "U.S.A.": "United States",
            "U.K.": "United Kingdom",
            "Czech": "Czech Republic",
            "Catarrh": "Qatar"
        }
        df_long['Country/Region'] = df_long['Country/Region'].replace(reemplazos_paises)

        # 13) Concatenate and classify by continent
        df_long['City & Country/ Region'] = df_long['City (ENG)'] + ', ' + df_long['Country/Region']
        df_long['Continent'] = df_long['Country/Region'].apply(obtener_continente)

        # 14) Add ISO country code and flag URL
        df_long['Country Code'] = df_long['Country/Region'].apply(obtener_codigo_iso)
        df_long['Flag URL'] = df_long['Country Code'].apply(
            lambda code: f"https://flagcdn.com/w40/{code.lower()}.png" if pd.notna(code) else None
        )

        # Save result
        df_long.to_excel(ruta_salida, index=False)
        print(f"✔ Saved: {ruta_salida}")

    except Exception as e:
        print(f"❌ Error processing {nombre_archivo}: {e}")

# Data Consolidation

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()
ruta_entrada = os.getenv("Carpeta_Normalizado")
ruta_salida  = os.getenv("Carpeta_Consolidada")
nombre_salida = "JLPT_Historico.xlsx"

if not ruta_entrada or not ruta_salida:
    raise RuntimeError("Error: Input or output paths are not defined in the .env file")

# Collect all DataFrames
dataframes = []
for archivo in os.listdir(ruta_entrada):
    # Skip OneDrive temporary files and any non-Excel files
    if not archivo.endswith(".xlsx") or archivo.startswith("~$"):
        continue

    ruta_archivo = os.path.join(ruta_entrada, archivo)
    try:
        df = pd.read_excel(ruta_archivo)
    except Exception as e:
        print(f"✘ Could not read '{archivo}': {e}")
        continue

    dataframes.append(df)

if not dataframes:
    raise RuntimeError("No valid files found for consolidation.")

# Concatenate without altering columns
df_consolidado = pd.concat(dataframes, ignore_index=True)

# Ensure 'Fecha' column is date only (no time)
if 'Fecha' in df_consolidado.columns:
    # Convert to datetime if not already, then extract date part only
    df_consolidado['Fecha'] = pd.to_datetime(df_consolidado['Fecha']).dt.date

# Ensure output folder exists
os.makedirs(ruta_salida, exist_ok=True)
ruta_salida_completa = os.path.join(ruta_salida, nombre_salida)

# Save the consolidated file
with pd.ExcelWriter(ruta_salida_completa, engine="openpyxl") as writer:
    df_consolidado.to_excel(writer, index=False, sheet_name="JLPT_Historico")
    # Auto-adjust column widths
    worksheet = writer.sheets["JLPT_Historico"]
    for col_cells in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in col_cells)
        col_letter = col_cells[0].column_letter
        worksheet.column_dimensions[col_letter].width = max_length + 2

print(f"✔ Consolidated file saved at: {ruta_salida_completa}")
